In [19]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests



In [20]:

# Check TensorFlow Version                                                                                                                                              
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU                                                                                                                                                       
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.12.0
Default GPU Device: /device:GPU:0


In [21]:
vgg_path = '/home/diogoaos/P2/CarND-Semantic-Segmentation/data/vgg'

In [22]:
def load_vgg(sess, vgg_path):
    """                                                                                                                                                                 
    Load Pretrained VGG Model into TensorFlow.                                                                                                                          
    :param sess: TensorFlow Session                                                                                                                                     
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"                                                                                   
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)                                                               
    """
    # TODO: Implement function                                                                                                                                          
    #   Use tf.saved_model.loader.load to load the model and weights                                                                                                    
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'                                                                                                                         
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    g = tf.get_default_graph()
    w1 = g.get_tensor_by_name(vgg_input_tensor_name)
    keep = g.get_tensor_by_name(vgg_keep_prob_tensor_name)
    l3 = g.get_tensor_by_name(vgg_layer3_out_tensor_name)
    l4 = g.get_tensor_by_name(vgg_layer4_out_tensor_name)
    l7 = g.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    
    return w1, keep, l3, l4, l7
    

In [23]:
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [24]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """                                                                                                                                                                 
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.                                                                       
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output                                                                                                             
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output                                                                                                             
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output                                                                                                             
    :param num_classes: Number of classes to classify                                                                                                                   
    :return: The Tensor for the last layer of output                                                                                                                    
    """
    # TODO: Implement function                                                                                                                                          
    return None
tests.test_layers(layers)

ValueError: None values not supported.

In [ ]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """                                                                                                                                                                 
    Train neural network and print out the loss during training.                                                                                                        
    :param sess: TF Session                                                                                                                                             
    :param epochs: Number of epochs                                                                                                                                     
    :param batch_size: Batch size                                                                                                                                       
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)                                                             
    :param train_op: TF Operation to train the neural network                                                                                                           
    :param cross_entropy_loss: TF Tensor for the amount of loss                                                                                                         
    :param input_image: TF Placeholder for input images                                                                                                                 
    :param correct_label: TF Placeholder for label images                                                                                                               
    :param keep_prob: TF Placeholder for dropout keep probability                                                                                                       
    :param learning_rate: TF Placeholder for learning rate                                                                                                              
    """
    # TODO: Implement function                                                                                                                                          
    pass
tests.test_train_nn(train_nn)